In [2]:
import numpy as np
import pandas as pd
import pymongo
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from config import config
from collections import defaultdict

def read_mongo(collection, query={}):
    """ Read from Mongo and Store into DataFrame """

    # Make a query to the specific DB and Collection
    cursor = collection.find(query)
    
    elements = []
    for doc in cursor:
        del doc['_id']
        elements.append(doc)
        
    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(elements)
    
    return df

def gerarLinha(repo, modelo, numeroDeIssuesDaRecomendacao):
    return {'projeto': repo, 'modelo': modelo, 'numeroDeIssuesDaRecomendacao': numeroDeIssuesDaRecomendacao, 'numeroDeIssuesTestas': 0, 'nTestes': 0, 'nRecomendacaoDadas': 0, 'totalAcertos': 0, 'totalErros': 0}
    

In [7]:
mongoClient = pymongo.MongoClient(config['DATABASE']['CONNECTION_STRING'])
db = mongoClient[config['DATABASE']['NAME']]
table = []
for col in db.list_collections():
  if not col['name'].endswith('_results'):
    continue
  # projeto (feito)
  # modelo (feito)
  # numeroDeIssuesDaRecomendacao (feito)
  # numeroDeIssuesTestas (feito)
  # nTestes(qnts arquivos corrigidos de verdade) (feito)
  # nRecomendacaoDadas(nTotalDeArquivos)
  # totalAcertos
  # totalErros
  repo = col['name'].replace('_results', '')
  if repo == 'microsoft/vscode':
    continue
  
  linhas = [
    gerarLinha(repo, 'sbert', 1),
    gerarLinha(repo, 'sbert', 3),
    gerarLinha(repo, 'sbert', 5),
    gerarLinha(repo, 'tfidf', 1),
    gerarLinha(repo, 'tfidf', 3),
    gerarLinha(repo, 'tfidf', 5),
    gerarLinha(repo, 'word2vec', 1),
    gerarLinha(repo, 'tfidf', 3),
    gerarLinha(repo, 'word2vec', 5),
  ]

  pos = {
    'sbert': 0,
    'tfidf': 3,
    'word2vec': 6
  }
  
  results = db[col['name']].find({'compare': 'title + body', 'filtros.daysBefore': 30})
  if not db[col['name']].count_documents({'compare': 'title + body', 'filtros.daysBefore': 30}):
    continue
  for result in results:
    tecnica = result['tecnica']
    posicao = pos[tecnica]
    if result['topk'] == 3:
      posicao += 1
    if result['topk'] == 5:
      posicao += 2
    
    linhas[posicao]['numeroDeIssuesTestas'] += 1
    linhas[posicao]['nTestes'] += len(result['arquivos_resolvidos_de_verdade'])
    linhas[posicao]['nRecomendacaoDadas'] += len(result['arquivos_sugeridos'])
    linhas[posicao]['totalAcertos'] += result['acertos']
    linhas[posicao]['totalErros'] += result['erros']
  
  table.extend(linhas)
  
table = pd.DataFrame(table)
table
  

,projeto,modelo,numeroDeIssuesDaRecomendacao,numeroDeIssuesTestas,nTestes,nRecomendacaoDadas,totalAcertos,totalErros
0,jabref/jabref,sbert,1,448,3175,2453,292,2161
1,jabref/jabref,sbert,3,448,3175,7252,365,6887
2,jabref/jabref,sbert,5,444,3163,12139,447,11692
3,jabref/jabref,tfidf,1,448,3175,2440,225,2215
4,jabref/jabref,tfidf,3,448,3175,7002,340,6662
5,jabref/jabref,tfidf,5,444,3163,12304,452,11852
6,jabref/jabref,word2vec,1,448,3175,3329,136,3193
7,jabref/jabref,tfidf,3,448,3175,8368,253,8115
8,jabref/jabref,word2vec,5,444,3163,13387,311,13076
9,tensorflow/tensorflow,sbert,1,349,936,636,45,591


In [8]:
mongoClient = pymongo.MongoClient(config['DATABASE']['CONNECTION_STRING'])
db = mongoClient[config['DATABASE']['NAME']]
table = []
for col in db.list_collections():
  if not col['name'].endswith('_results'):
    continue
  # projeto (feito)
  # modelo (feito)
  # numeroDeIssuesDaRecomendacao (feito)
  # numeroDeIssuesTestas (feito)
  # nTestes(qnts arquivos corrigidos de verdade) (feito)
  # nRecomendacaoDadas(nTotalDeArquivos)
  # totalAcertos
  # totalErros
  repo = col['name'].replace('_results', '')
  if repo == 'microsoft/vscode':
    continue
  
  linhas = [
    gerarLinha(repo, 'sbert', 1),
    gerarLinha(repo, 'sbert', 3),
    gerarLinha(repo, 'sbert', 5),
    gerarLinha(repo, 'tfidf', 1),
    gerarLinha(repo, 'tfidf', 3),
    gerarLinha(repo, 'tfidf', 5),
    gerarLinha(repo, 'word2vec', 1),
    gerarLinha(repo, 'tfidf', 3),
    gerarLinha(repo, 'word2vec', 5),
  ]

  pos = {
    'sbert': 0,
    'tfidf': 3,
    'word2vec': 6
  }
  
  results = db[col['name']].find({'compare': 'title + body', 'filtros.daysBefore': 180})
  if not db[col['name']].count_documents({'compare': 'title + body', 'filtros.daysBefore': 180}):
    continue
  for result in results:
    tecnica = result['tecnica']
    posicao = pos[tecnica]
    if result['topk'] == 3:
      posicao += 1
    if result['topk'] == 5:
      posicao += 2
    
    linhas[posicao]['numeroDeIssuesTestas'] += 1
    linhas[posicao]['nTestes'] += len(result['arquivos_resolvidos_de_verdade'])
    linhas[posicao]['nRecomendacaoDadas'] += len(result['arquivos_sugeridos'])
    linhas[posicao]['totalAcertos'] += result['acertos']
    linhas[posicao]['totalErros'] += result['erros']
  
  table.extend(linhas)
  
table = pd.DataFrame(table)
table
  

,projeto,modelo,numeroDeIssuesDaRecomendacao,numeroDeIssuesTestas,nTestes,nRecomendacaoDadas,totalAcertos,totalErros
0,jabref/jabref,sbert,1,448,3175,2711,268,2443
1,jabref/jabref,sbert,3,448,3175,7456,445,7011
2,jabref/jabref,sbert,5,448,3175,12162,574,11588
3,jabref/jabref,tfidf,1,448,3175,2373,254,2119
4,jabref/jabref,tfidf,3,448,3175,6966,402,6564
5,jabref/jabref,tfidf,5,448,3175,11949,471,11478
6,jabref/jabref,word2vec,1,448,3175,2929,56,2873
7,jabref/jabref,tfidf,3,448,3175,8943,200,8743
8,jabref/jabref,word2vec,5,448,3175,15335,279,15056
9,tensorflow/tensorflow,sbert,1,353,944,695,64,631
